## Imports

In [1]:
from pymatgen.transformations.standard_transformations import AutoOxiStateDecorationTransformation
import time
import numpy as np
from pymatgen.core.periodic_table import Species
import pandas as pd
from pymatgen.core.structure import Structure

## Class to add oxidation states to structure

In [2]:
aosdt = AutoOxiStateDecorationTransformation()

## Get testing structure and add oxidation states

In [3]:
structure = Structure.from_file('mp-4681.vasp')
oxi_structure = aosdt.apply_transformation(structure)

In [4]:
species = np.unique([s.specie for s in oxi_structure])
oxi_dict = {}
for s in species:
    oxi_dict[str(s.element)] = s.oxi_state

## Compute Matminer GII - Benchmarking

In [5]:
from matminer.featurizers.structure.bonding import GlobalInstabilityIndex
# matminer GlobalInstabilityIndex uses bvparm16.cif, which is the default for GIICalculator

In [6]:
start_time = time.time()
mm_gii_calc = GlobalInstabilityIndex(r_cut=4)
mm_gii = mm_gii_calc.featurize(oxi_structure)[0]
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.4463930130004883 seconds ---


In [7]:
mm_gii

0.2295876515555516

## Compute GIICalculator GII

In [8]:
from gii_minimization import GIICalculator

In [9]:
start_time = time.time()
#gii_calc = GIICalculator(method='CrystalNN')
gii_calc = GIICalculator(method='Cutoff', cutoff=4)
gii = gii_calc.GII(oxi_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.09987139701843262 seconds ---


In [10]:
gii

0.22958765155555164

In [11]:
gii_calc.params_dict

{'Cation': [Species Na+, Species Nb5+],
 'Anion': [Species O2-, Species O2-],
 'R0': [1.803, 1.911],
 'B': [0.37, 0.37]}

## Perform the Site Optimization

In [12]:
from gii_minimization import SiteClusterOptimization

In [13]:
sco = SiteClusterOptimization()

In [14]:
start_time = time.time()
final_structure = sco.minimize_cluster_di_squared(oxi_structure, site_ind=16) # pass the index of site to be optimized
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 1.5238230228424072 seconds ---


In [15]:
start_time = time.time()
start_gii = gii_calc.GII(oxi_structure)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.07769083976745605 seconds ---


In [16]:
start_gii

0.22958765155555164

In [17]:
start_time = time.time()
final_gii = gii_calc.GII(final_structure, use_sym=False)
print("Time taken: --- %s seconds ---" % (time.time() - start_time))

Time taken: --- 0.18429183959960938 seconds ---


In [18]:
final_gii

0.22521258089793017

## Perform the Structure Optimization 

In [19]:
from gii_minimization import GIIMinimizer

In [21]:
gii_minimizer = GIIMinimizer(oxi_structure, convergence_tolerance=0.01)
# All sites optimized until optimization of said site changes GII by < convergence_tolerance

In [22]:
optimized_structure = gii_minimizer.gii_minimization(opt_method='max')

Step 1 complete; 0.22958765155555255 --> 0.21549777952098717
Step 2 complete; 0.21549777952098717 --> 0.19173268867430582
Step 3 complete; 0.19173268867430582 --> 0.1804305852801663
Step 4 complete; 0.1804305852801663 --> 0.15846711462195934
Step 5 complete; 0.15846711462195934 --> 0.15270143025604774
Step 6 complete; 0.15270143025604774 --> 0.14251610265431833
Step 7 complete; 0.14251610265431833 --> 0.12463795328881057
Step 8 complete; 0.12463795328881057 --> 0.12089018037407909
Step 9 complete; 0.12089018037407909 --> 0.10077945970741657
Step 10 complete; 0.10077945970741657 --> 0.08919326280330687
Step 11 complete; 0.08919326280330687 --> 0.08919326280330686
Step 12 complete; 0.08919326280330686 --> 0.07832052207686163
Step 13 complete; 0.07832052207686163 --> 0.07832052207686163
Step 14 complete; 0.07832052207686163 --> 0.07832052207686165
Step 15 complete; 0.07832052207686165 --> 0.07832052207686165
Step 16 complete; 0.07832052207686165 --> 0.07832052207686171
Step 17 complete; 0

In [27]:
# Get the change in GII following optimization for each site in the structure, show that it is < convergence_tolerance
for i in range(len(gii_minimizer.diffs)):
    print('Specie: %s, delGII of optimization step: %s' % (optimized_structure[i].specie, gii_minimizer.diffs[i]))

Specie: Na+, delGII of optimization step: 0.0038284613437287035
Specie: Na+, delGII of optimization step: 0.007000995584628057
Specie: Na+, delGII of optimization step: 0.005472528375726302
Specie: Na+, delGII of optimization step: 0.006260648810838057
Specie: Nb5+, delGII of optimization step: -6.938893903907228e-17
Specie: Nb5+, delGII of optimization step: 0.0
Specie: Nb5+, delGII of optimization step: -1.3877787807814457e-17
Specie: Nb5+, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.0010099482324391826
Specie: O2-, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 9.351138524076041e-05
Specie: O2-, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.0037477729147314814
Specie: O2-, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.0023288005630781594
Specie: O2-, delGII of optimization step: 0.0
Specie: O2-, delGII of optimization step: 0.0057656843659116
Specie: O2-, delGII